In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn
import pickle

import os

os.environ['PYTHONHASHSEED'] = str(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(42)
np.random.seed(42)
rn.seed(42)

In [2]:
train_dir = './data/X_train.pickle'
train_label_dir = './data/data_y_train.csv'
test_dir = './data/X_test.pickle'
test_label_dir = './data/data_y_test.csv'

with open(train_dir,'rb') as frb:
    X_train = pickle.load(frb) # train_feature_load

with open(test_dir,'rb') as frb:
    X_test = pickle.load(frb) # train_feature_load

X_train = np.array(X_train)[:,:,2:]
X_test = np.array(X_test)[:,:,2:]
y_train = pd.read_csv(train_label_dir)['label']  # train_label load
y_test = pd.read_csv(test_label_dir)['label']  # train_label load

In [3]:
from tensorflow.keras import layers, initializers
from tensorflow.keras.models import Model

In [4]:
filter_1 = 19
filter_2 = 19
filter_3 = 43
channel_1 = 128
channel_2 = 256
channel_3 = 128
input_channel = 8
stride = 2
drop_rate = 0.3

inputs = layers.Input(shape=(600,input_channel))

conv = layers.Conv1D(
    channel_1,filter_1,strides=stride,padding='same',activation='relu',
    kernel_initializer=initializers.he_normal(seed=42))(inputs)
conv = layers.BatchNormalization()(conv)
conv = layers.Dropout(drop_rate,seed=42)(conv)

conv = layers.Conv1D(
    channel_2,filter_2,strides=stride,padding='same',activation='relu',
    kernel_initializer=initializers.he_normal(seed=42))(conv)
conv = layers.BatchNormalization()(conv)
conv = layers.Dropout(drop_rate,seed=42)(conv)

conv = layers.Conv1D(
    channel_3,filter_3,strides=stride,padding='same',activation='relu',
    kernel_initializer=initializers.he_normal(seed=42))(conv)
conv = layers.BatchNormalization()(conv)
conv = layers.Dropout(drop_rate,seed=42)(conv)

pool = layers.GlobalAveragePooling1D()(conv)
pool = layers.BatchNormalization()(pool)

outputs = layers.Dense(61,activation='softmax',kernel_initializer=initializers.he_normal(seed=42))(pool)
model = Model(inputs,outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 600, 8)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 128)          19584     
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 128)          512       
_________________________________________________________________
dropout (Dropout)            (None, 300, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 150, 256)          622848    
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 256)          1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 256)          0     

In [5]:
checkpoint_filepath = "./save/simple_cnn_best.hdf5"

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(patience = 5,verbose = 1,factor = 0.5) 

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',min_delta=0.0001,
    patience=30,verbose=1)

history = model.fit(
    X_train[:,:,:input_channel],y_train,
    epochs=2000,
    callbacks=[save_best,early_stop,lr_scheduler],
    validation_split=0.2
    )

Epoch 1/2000
63/63 [==============================] - 4s 21ms/step - loss: 2.7584 - accuracy: 0.4560 - val_loss: 2.5592 - val_accuracy: 0.5020

Epoch 00001: val_loss improved from inf to 2.55925, saving model to ./save/simple_cnn_best.hdf5
Epoch 2/2000
63/63 [==============================] - 1s 16ms/step - loss: 1.9199 - accuracy: 0.5735 - val_loss: 2.7983 - val_accuracy: 0.5200

Epoch 00002: val_loss did not improve from 2.55925
Epoch 3/2000
63/63 [==============================] - 1s 16ms/step - loss: 1.4781 - accuracy: 0.6265 - val_loss: 3.1699 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 2.55925
Epoch 4/2000
63/63 [==============================] - 1s 16ms/step - loss: 1.2330 - accuracy: 0.6820 - val_loss: 3.3224 - val_accuracy: 0.5020

Epoch 00004: val_loss did not improve from 2.55925
Epoch 5/2000
63/63 [==============================] - 1s 16ms/step - loss: 1.0431 - accuracy: 0.7220 - val_loss: 3.7095 - val_accuracy: 0.4840

Epoch 00005: val_loss did not i

In [6]:
model.load_weights(checkpoint_filepath)
model.evaluate(X_test[:,:,:input_channel],y_test)


20/20 [==============================] - 0s 5ms/step - loss: 0.5941 - accuracy: 0.8288


[0.5940622687339783, 0.8288000226020813]